In [ ]:
import os
import pandas as pd
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input
import numpy as np 

def get_encoder_decoder(model_path):
    model = load_model(model_path)
    
    encoder_inputs = model.input[0]  # input_1
 
    encoder_lstm = model.get_layer('Encoder-LSTM')
    latent_dim = encoder_lstm.output_shape[-1][1] 
    encoder_outputs, state_h_enc, state_c_enc = encoder_lstm.output  # lstm_1
    
    encoder_states = [state_h_enc, state_c_enc]
    encoder_model =  Model(encoder_inputs, encoder_states,name='Encoder-Model')

    decoder_inputs = model.input[1]  # input_2
    decoder_state_input_h = Input(shape=(latent_dim,))
    decoder_state_input_c = Input(shape=(latent_dim,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

    decoder_lstm = model.get_layer('Decoder-LSTM')
    decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
    
    decoder_states = [state_h_dec, state_c_dec]
    decoder_dense = model.get_layer('Decoder-Dense')
    decoder_outputs = decoder_dense(decoder_outputs)

    decoder_model = Model( [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states,name='Decoder-Model')
    return encoder_model, decoder_model 


class Decoder:
    def __init__(self, encoder_model, decoder_model, input_token2id,target_token2id, P):
        
        self.encoder_model = encoder_model
        self.decoder_model = decoder_model 
        self.latent_dim = P['latent_dim']
        self.num_decoder_tokens = P['num_decoder_tokens']
        self.max_decoder_seq_length = P['max_decoder_seq_length']

        self.input_token2id =  input_token2id
        self.target_token2id = target_token2id 

        #print("targte_token2id", self.target_token2id)
        
        self.input_id2toekn = dict((i, char) for char, i in self.input_token2id.items())
        self.target_id2token = dict((i, char) for char, i in self.target_token2id.items())
      
    def decode_sequence(self, input_seq):
        # Encode the input as state vectors.
        states_value = self.encoder_model.predict(input_seq)

        print("self.target_token_index",self.target_token2id["\t"])
        # Generate empty target sequence of length 1.
        target_seq = np.zeros((1, 1, self.num_decoder_tokens))
        # Populate the first character of target sequence with the start character.
        target_seq[0, 0, self.target_token2id["\t"]] = 1.0

        # Sampling loop for a batch of sequences
        # (to simplify, here we assume a batch of size 1).
        stop_condition = False
        decoded_sentence = ""
        while not stop_condition:
            output_tokens, h, c = self.decoder_model.predict([target_seq] + states_value)
        
            # Sample a token
            sampled_token_index = np.argmax(output_tokens[0, -1, :])
            sampled_char = self.target_id2token[sampled_token_index]
            decoded_sentence += sampled_char

            # Exit condition: either hit max length
            # or find stop character.
            if sampled_char == "\n" or len(decoded_sentence) > self.max_decoder_seq_length:
                stop_condition = True

            # Update the target sequence (of length 1).
            target_seq = np.zeros((1, 1, self.num_decoder_tokens))
            target_seq[0, 0, sampled_token_index] = 1.0

            # Update states
            states_value = [h, c]
        return decoded_sentence





if __name__ == "__main__":

    workspace_dir= r"C:\Users\jayanti.prasad\Projects-Dev\Seq2Seq\seq2seq_char1\tmp"
    model_path = r"C:\Users\jayanti.prasad\Projects-Dev\Seq2Seq\seq2seq_char1\tmp\trained_model\model.hdf5"

    df_config = pd.read_csv(workspace_dir + os.sep + "params.csv")
    df_vocab_in = pd.read_csv(workspace_dir + os.sep + "vocab_in.csv",encoding='utf-8')
    df_vocab_out = pd.read_csv(workspace_dir + os.sep + "vocab_out.csv",encoding='utf-8')

    
    P = dict (zip (df_config['key'].to_list(), df_config['value'].to_list()))


    inp_tokens = df_vocab_in['token'].to_list()
    out_tokens = df_vocab_out['token'].to_list() 
    
    input_token2id = dict (zip (inp_tokens, [i for i in range (0, len (inp_tokens))]))
    target_token2id = dict (zip (out_tokens, [i for i in range (0, len (out_tokens))]))
       
    encoder_model, decoder_model  = get_encoder_decoder(model_path)

    print(encoder_model.summary())
    print(decoder_model.summary())


    # get some input data for decoding 
    data_path = r"C:\Users\jayanti.prasad\Data\NLP_DATA\seq2seq_data\fra.txt"
    with open(data_path, "r", encoding="utf-8") as f:
         lines = f.read().split("\n")
      
    input_texts = []
    target_texts = []
    num_samples = 2000 
    
    for line in lines[: min(num_samples, len(lines) - 1)]:
         raw_line = line.split("\t")
         #input_text, target_text, _ = line.split("\t")
         input_text, target_text = raw_line[0], raw_line[1] 
         target_text = "\t" + target_text + "\n"
         input_texts.append(input_text)
         target_texts.append(target_text)

    encoder_input_data = np.zeros((len(input_texts), P['max_encoder_seq_length'], P['num_encoder_tokens']), dtype="float32")
    
    for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
        for t, char in enumerate(input_text):
            encoder_input_data[i, t, input_token2id[char]] = 1.0
            encoder_input_data[i, t + 1 :, input_token2id[" "]] = 1.0
     

    D = Decoder (encoder_model, decoder_model, input_token2id,target_token2id, P) 
         
    for seq_index in range(20):
         # Take one sequence (part of the training set)
         # for trying out decoding.
         input_seq = encoder_input_data[seq_index : seq_index + 1]
         decoded_sentence = D.decode_sequence(input_seq)
         print("-")
         print("Input sentence:", input_texts[seq_index])
         print("Decoded sentence:", decoded_sentence)

Model: "Encoder-Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Encoder-Input (InputLayer)  [(None, None, 71)]        0         
                                                                 
 Encoder-LSTM (LSTM)         [(None, 60),              31680     
                              (None, 60),                        
                              (None, 60)]                        
                                                                 
Total params: 31680 (123.75 KB)
Trainable params: 31680 (123.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Model: "Decoder-Model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 Decoder-Input (InputLayer)  [(None, None, 94)]      